In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
import os
os.environ['HF_HOME'] = './cache'
print("huggingface cache is in {}".format(os.getenv('HF_HOME')))

huggingface cache is in ./cache


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/scratch/kqa3/anaconda3/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
base_model = 'NousResearch/Llama-2-7b-chat-hf'

In [5]:
# Reload model in FP16 and merge it with LoRA weights
load_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
    # some paramerters from https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035/3
    temperature=1.0,
    # do_sample=True,gggg

)

# model = PeftModel.from_pretrained(load_model, new_model)
model = PeftModel.from_pretrained(load_model)
# model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.20it/s]
/scratch/kqa3/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/scratch/kqa3/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corres

TypeError: PeftModel.from_pretrained() missing 1 required positional argument: 'model_id'

In [7]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [8]:
# Run text generation pipeline with our next model
prompt = "Who is Leonardo Da Vinci?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] the account of a user who had voted on your question has been deleted. sometimes users do this themselves, rarely we have to do it for them. when they delete their account, everything they touched goes away with them. see also what does &quot;user was removed&quot; mean and why did my reputation change because of it?.


In [9]:
prompt = "What is result of 1+1?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is result of 1+1? [/INST] this is not a security issue. this is a programming issue. the code is not doing what you want, so you are asking where the code is going wrong. from the looks of it, the code is following the logic that you have provided. you want the code to do something different. this is not a security issue, this is a programming issue. you are getting unexpected results because the code is following the logic that you have provided. this is not a security issue, this is a programming issue. your question is not about security, but about how your code is not being followed. your question is not about security, but about how your code is not being followed. your question is not about security, but about how your code is not being followed. your question is not about security, but about how your code is not being followed. your question is not about security, but about how your code is


In [10]:
prompt = "What's shellcode? and what's relative addressing technique when you want to implement spwning a shell?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What's shellcode? and what's relative addressing technique when you want to implement spwning a shell? [/INST] "nuclear option" is a metaphor for wiping all local storage, overwriting all firmware, and physically disabling the device. this is useful for preventing attackers from using the device for malicious activities after a compromise, but it is also a lot of damage for no apparent reason. why would a security professional want to destroy devices they are servicing? if an attacker is able to gain remote access to a device, wiping its local storage and disabling it is a lot easier than physically disabling the device. if an attacker can physically access the device, disabling it permanently is a lot easier than trying to recover it after a compromise. if you are in a position where you are servicing devices and destroying them, you should be


In [14]:
prompt = "what is a hacker?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] what is a hacker? [/INST] the hacker is someone who appreciates the aesthetics of broken things. often, this is applied to computer systems, where a hacker might enjoy exploring the details of an exploit, or a backdoor, or an elevated privilege level. but in the information security field, we are also exposed to broken things on a daily basis, whether it is a misconfigured server, or a policy that needs tweaking, or a desperate user who is frustrated with the state of security in the world. the hacker, then, is someone who is comfortable with the complex relationships between people, systems, and information, and who enjoys exploring the aesthetics of broken things, and finding the hidden beauty in the complex relationships between technology and society. so what does this mean in practical terms? it means that when we are discussing a particularly nasty piece of malware


In [19]:
prompt = "Can you use GPG to decrypt anbd encrypt with a different key in one step?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Can you use GPG to decrypt anbd encrypt with a different key in one step? [/INST] the encryption and decryption are performed separately. the encryption/decryption algorithm is based on aes and aes-cmc. the encryption/decryption key is derived from the user's password. the user can set a very long (but reasonably short) aes key, like 30-40 digits, unchanged during runtime. the user can set an identical, very long, fixed aes-cmc key. the user can set an identical, very long fixed aes key for encryption and decryption. the user can set a very long, fixed aes key for the aes-cmc. the user can set a very long, fixed aes key for the encryption and decryption. the user can set a very long, fixed aes key for the aes-cmc. the user can set


In [20]:
prompt = "How to securely store secrets in Docker container?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How to securely store secrets in Docker container? [/INST] 140 members of security professionals from around the world gathered in dubai for the esfu security forum. the event was held under the patronage of his excellency sheikh hamdan bin mubarak al-mu’alla, the crown prince of dubai, and in partnership with etisalat. the conference addressed issues related to physical security, information security, risk management and other aspects of security. some of the highlights: keynote speaker: mohammed al-muhairi, uae national and ceo of etisalat speaking session: industry experts including mohammed al-muhairi, rory mccoy, steve hurn, and others breakout sessions: physical security, information security, risk management and other topics panel discussions: cyberterrorism; the future of physical security; the future
